# 01. 데이터 스캔 (Data Scan)

SAPA 데이터 구조를 파악하고, 기본 정보를 확인합니다.

## 학습 목표
- SAPA 데이터의 구조 이해
- Planned missingness 개념 이해
- 인구통계 변수와 성격 문항 구분

In [1]:
# 필요한 라이브러리 설치 (처음 한 번만 실행)
%pip install pandas numpy -q


[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
import os

# 상위 폴더로 이동해서 데이터 접근
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
print(f'작업 폴더: {os.getcwd()}')

작업 폴더: /Users/serinoh/SAPA


## 1. 데이터 로드

In [ ]:
# SAPA 응답 데이터 로드
df = pd.read_csv('data/raw/sapa_data.csv', encoding='utf-8')
print(f'데이터 크기: {df.shape}')
print(f'응답자 수: {len(df):,}')
print(f'변수 수: {len(df.columns)}')

데이터 크기: (23679, 719)
응답자 수: 23,679
변수 수: 719


In [ ]:
# 채점 키 로드
keys = pd.read_csv('data/raw/superKey696.csv', index_col=0, encoding='utf-8')
print(f'채점 키 크기: {keys.shape}')
print(f'문항 수: {len(keys)}')
print(f'척도 수: {len(keys.columns)}')

채점 키 크기: (696, 131)
문항 수: 696
척도 수: 131


## 2. 변수 구분

SAPA 데이터는 **인구통계 변수**와 **성격 문항**으로 구성됩니다.

In [5]:
# 인구통계 변수 (22개)
demo_cols = ['gender', 'relstatus', 'age', 'marstatus', 'height', 'BMI', 
             'weight', 'exer', 'smoke', 'country', 'state', 'ethnic', 'education',
             'jobstatus', 'occPrestige', 'occIncomeEst', 'p1edu', 'p1occPrestige',
             'p1occIncomeEst', 'p2edu', 'p2occPrestige', 'p2occIncomeEst']

# 성격 문항 (q_로 시작)
item_cols = [col for col in df.columns if col.startswith('q_')]

print(f'인구통계 변수: {len(demo_cols)}개')
print(f'성격 문항: {len(item_cols)}개')

인구통계 변수: 22개
성격 문항: 696개


## 3. Planned Missingness 확인

SAPA는 **planned missingness** 설계입니다:
- 각 참가자가 전체 696문항 중 **일부만** 응답
- 평균 약 76문항 응답
- 결측률 약 89% → **정상!**

In [6]:
# 결측률 계산
item_missing_rate = df[item_cols].isnull().mean().mean()
print(f'문항 전체 결측률: {item_missing_rate:.1%}')

문항 전체 결측률: 87.6%


In [7]:
# 응답자당 평균 응답 문항 수
responses_per_person = df[item_cols].notna().sum(axis=1)
print(f'응답자당 평균 응답 문항: {responses_per_person.mean():.0f}개')
print(f'범위: {responses_per_person.min()} ~ {responses_per_person.max()}개')

응답자당 평균 응답 문항: 86개
범위: 0 ~ 311개


## 4. 채점 키 구조

`superKey696.csv`는 점수 계산에 사용하는 **채점 키 매트릭스**입니다.
- `1`: 정채점
- `-1`: 역채점 (7 - 원점수)
- `0`: 해당 안됨

In [8]:
# 채점 키 주요 척도
print('=== 주요 척도 ===')
print('NEO Big Five:', [col for col in keys.columns if col.startswith('NEO_')])
print('HEXACO:', [col for col in keys.columns if col.startswith('HEXACO_')])

=== 주요 척도 ===
NEO Big Five: ['NEO_A', 'NEO_C', 'NEO_E', 'NEO_O', 'NEO_N']
HEXACO: ['HEXACO_A', 'HEXACO_C', 'HEXACO_X', 'HEXACO_O', 'HEXACO_E', 'HEXACO_H']


In [9]:
# NEO_E (외향성) 척도의 문항 확인
neo_e_items = keys.index[keys['NEO_E'] != 0].tolist()
print(f'NEO Extraversion 문항 수: {len(neo_e_items)}')
print(f'문항 예시: {neo_e_items}')

NEO Extraversion 문항 수: 60
문항 예시: ['q_20', 'q_31', 'q_71', 'q_74', 'q_140', 'q_175', 'q_200', 'q_294', 'q_296', 'q_313', 'q_314', 'q_429', 'q_455', 'q_505', 'q_576', 'q_595', 'q_684', 'q_690', 'q_744', 'q_745', 'q_746', 'q_799', 'q_819', 'q_1043', 'q_1114', 'q_1151', 'q_1169', 'q_1180', 'q_1196', 'q_1244', 'q_1248', 'q_1280', 'q_1281', 'q_1329', 'q_1330', 'q_1350', 'q_1360', 'q_1368', 'q_1370', 'q_1371', 'q_1376', 'q_1410', 'q_1480', 'q_1531', 'q_1567', 'q_1599', 'q_1600', 'q_1662', 'q_1664', 'q_1666', 'q_1670', 'q_1685', 'q_1768', 'q_1769', 'q_1803', 'q_1872', 'q_1913', 'q_1923', 'q_1943', 'q_2017']


In [10]:
# 역채점 문항 확인
reverse_items = keys.index[keys['NEO_E'] == -1].tolist()
print(f'NEO_E 역채점 문항: {len(reverse_items)}개')
print(reverse_items)

NEO_E 역채점 문항: 25개
['q_140', 'q_175', 'q_200', 'q_313', 'q_314', 'q_576', 'q_684', 'q_690', 'q_1114', 'q_1151', 'q_1180', 'q_1196', 'q_1280', 'q_1281', 'q_1329', 'q_1330', 'q_1480', 'q_1531', 'q_1567', 'q_1600', 'q_1666', 'q_1685', 'q_1913', 'q_1923', 'q_2017']


## 5. 데이터 샘플 확인

In [11]:
# 첫 5명의 인구통계 정보
df[['RID', 'gender', 'age', 'education', 'state']].head()

,RID,gender,age,education,state
0,111610,female,19to24,CurrentInUniv,other
1,236351,female,18andUnder,less12yrs,other
2,258633,male,18andUnder,NaN,other
3,273126,female,18andUnder,NaN,other
4,371933,female,19to24,CurrentInUniv,other


In [12]:
# 첫 5명의 성격 문항 응답 (처음 10문항)
df[item_cols[:10]].head()

,q_6,q_20,q_22,q_23,q_26,q_31,q_35,q_38,q_39,q_40
0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,2.0
1,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
2,NaN,5.0,NaN,4.0,3.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
